# Creating Dashboard with Python

## Import Libraries

In [1]:
# Import libraries
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from datetime import datetime as dt
from streamlit_keplergl import keplergl_static

## Wrangle Data

In [2]:
# Import data
df = pd.read_csv('Data/Prepared Data/NY_merged_agg.csv', index_col = 0)

In [3]:
# Checking data types
df.dtypes

ride_id                object
rideable_type          object
start_time             object
end_time               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
date                   object
avgTemp_C             float64
avgTemp_F             float64
daily_rides             int64
_merge                 object
dtype: object

In [4]:
# Creating a month column 
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df['month'] = df['date'].dt.month
df['month'] = df['month'].astype('int')

In [5]:
# Creating a season column
df['season'] = [
"winter" if (month == 12 or 1 <= month <= 4)
    else "spring" if (4 < month <= 5)
    else "summer" if (6 <= month <= 9)
    else "fall"
for month in df['month']
    ]

In [6]:
# Creating a map from month integers to names
month_name_mapping = {
    1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr',
    5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug',
    9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}
# Add new month_name column
df['month_name'] = df['month'].map(month_name_mapping)

In [7]:
# Checking dataframe
df.head()

,ride_id,rideable_type,start_time,end_time,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,date,avgTemp_C,avgTemp_F,daily_rides,_merge,month,season,month_name
0,115C78C3039FFA89,electric_bike,2022-01-01 09:21:14,2022-01-01 09:35:46,Essex Light Rail,JC038,Essex Light Rail,JC038,40.712774,-74.036486,...,-74.036486,member,2022-01-01,11.6,52.88,592,both,1,winter,Jan
1,7FFD810CAA7A919E,classic_bike,2022-01-01 02:43:56,2022-01-01 02:43:57,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,...,-74.024020,member,2022-01-01,11.6,52.88,592,both,1,winter,Jan
2,E715E8432031B72C,classic_bike,2022-01-01 02:13:33,2022-01-01 02:18:42,Essex Light Rail,JC038,Washington St,JC098,40.712774,-74.036486,...,-74.035483,member,2022-01-01,11.6,52.88,592,both,1,winter,Jan
3,BF1B7B1E1961A87B,electric_bike,2022-01-01 17:18:46,2022-01-01 18:55:25,Grand St,JC102,W 27 St & 7 Ave,6247.06,40.715178,-74.037683,...,-73.993915,casual,2022-01-01,11.6,52.88,592,both,1,winter,Jan
4,4A01F0E53C6F4386,electric_bike,2022-01-01 11:23:32,2022-01-01 11:29:27,Christ Hospital,JC034,Hoboken Terminal - Hudson St & Hudson Pl,HB101,40.734786,-74.050444,...,-74.030305,member,2022-01-01,11.6,52.88,592,both,1,winter,Jan


## Bar Charts with Plotly

In [8]:
# Grouping by start station and creating dataframe of top 20 most popular stations
df['value'] = 1 
df_group = df.groupby('start_station_name', as_index=False).agg({'value': 'sum'})
top20 = df_group.nlargest(20, 'value')

In [ ]:
# Creating bar chart of top 20 stations
fig = go.Figure(go.Bar(x = top20['start_station_name'], y = top20['value'], marker={'color': top20['value'],'colorscale': 'viridis_r'}))
fig.show()

In [ ]:
# Adding details to bar chart
fig.update_layout(
    title = 'Top 20 Most Popular Bike Stations in New York',
    xaxis_title = 'Start stations',
    yaxis_title ='Total trips',
    width = 900, height = 600
)

In [18]:
# Exporting top20 dataframe to .csv
top20.to_csv('top20.csv')

## Line Chart in Plotly

In [21]:
# Creating a 7-day rolling average to smooth line chart
df['daily_rides_rolling'] = df['daily_rides'].rolling(window=14).mean()
df['avgTemp_F_rolling'] = df['avgTemp_F'].rolling(window=14).mean()

In [ ]:
# Creating line chart

fig_2 = make_subplots(specs = [[{"secondary_y": True}]])

# Original daily rides trace
fig_2.add_trace(
go.Scatter(x = df['date'], y = df['daily_rides'], name = 'Daily Bike Rides',
           marker={'color': df['daily_rides'],'color': 'navy'}),
    secondary_y = False
)

# Add 7-day rolling average for Daily Bike Rides
fig_2.add_trace(
go.Scatter(x=df['date'], y=df['daily_rides_rolling'], name='7-Day Avg: Daily Bike Rides',
        line={'color': 'navy', 'dash': 'dash'}),
    secondary_y=False
)

# Original temperature trace
fig_2.add_trace(
go.Scatter(x=df['date'], y = df['avgTemp_F'], name = 'Daily Temperature',
           marker={'color': df['avgTemp_F'],'color': 'red'}),
    secondary_y=True
)

# Add 7-day rolling average for Daily Temperature
fig_2.add_trace(
go.Scatter(x=df['date'], y=df['avgTemp_F_rolling'], name='7-Day Avg: Daily Temperature',
        line={'color': 'red', 'dash': 'dash'}
    ),
    secondary_y=True
)

In [ ]:
# Adding details to line chart
fig_2.update_layout(
    title = 'Daily Bike Trips and Temperature in New York 2022',
    xaxis_title = 'Month',
    yaxis=dict(
        title="Total Rides",
        titlefont=dict(color="navy"),
        tickfont=dict(color="navy")  
    ),
    yaxis2=dict(
        title="Average Temperature (\u00B0F)",
        titlefont=dict(color="red"),
        tickfont=dict(color="red"),
        overlaying="y",
        side="right"
    ),
    width = 1100, height = 400
)

## Reducing Data

In [22]:
df.tail()

,ride_id,rideable_type,start_time,end_time,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,avgTemp_C,avgTemp_F,daily_rides,_merge,month,season,month_name,value,daily_rides_rolling,avgTemp_F_rolling
895480,1C2DA98EA3E22C36,electric_bike,2022-12-31 11:04:03,2022-12-31 11:07:05,Southwest Park - Jackson St & Observer Hwy,HB401,City Hall - Washington St & 1 St,HB105,40.737551,-74.041664,...,8.2,46.76,983,both,12,winter,Dec,1,983.0,46.76
895481,15CAD9793CBB48C2,classic_bike,2022-12-31 09:58:06,2022-12-31 10:11:04,Riverview Park,JC057,Manila & 1st,JC082,40.744703,-74.043759,...,8.2,46.76,983,both,12,winter,Dec,1,983.0,46.76
895482,D48F644880B431D1,classic_bike,2022-12-31 16:35:25,2022-12-31 17:00:09,Manila & 1st,JC082,Riverview Park,JC057,40.721651,-74.042884,...,8.2,46.76,983,both,12,winter,Dec,1,983.0,46.76
895483,1E992E823B5058B7,classic_bike,2022-12-31 16:05:25,2022-12-31 16:12:24,City Hall - Washington St & 1 St,HB105,11 St & Washington St,HB502,40.737360,-74.030970,...,8.2,46.76,983,both,12,winter,Dec,1,983.0,46.76
895484,055D6A3EC0CD3E03,classic_bike,2022-12-31 18:00:20,2022-12-31 18:11:33,11 St & Washington St,HB502,City Hall - Washington St & 1 St,HB105,40.749985,-74.027150,...,8.2,46.76,983,both,12,winter,Dec,1,983.0,46.76


In [23]:
# Checking columns in dataframe
df.columns

Index(['ride_id', 'rideable_type', 'start_time', 'end_time',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'date', 'avgTemp_C', 'avgTemp_F', 'daily_rides',
       '_merge', 'month', 'season', 'month_name', 'value',
       'daily_rides_rolling', 'avgTemp_F_rolling'],
      dtype='object')

In [24]:
# Removing columns to keep only those necessary for dashboard
df_1 = df.drop(columns = {'ride_id', 'rideable_type', 'start_time', 'end_time', 'start_station_id', 'end_station_id',
                          'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual', 'avgTemp_C', '_merge', 'month'})

In [25]:
# Checking columns of new dataframe
df_1.columns

Index(['start_station_name', 'end_station_name', 'date', 'avgTemp_F',
       'daily_rides', 'season', 'month_name', 'value', 'daily_rides_rolling',
       'avgTemp_F_rolling'],
      dtype='object')

In [26]:
# Exporting reduced dataset
df_1.to_csv('Data/Prepared Data/reduced_data.csv')

## Creating Random Split

In [27]:
df_1.shape

(895485, 10)

In [28]:
# Creating random seed
np.random.seed(32)
red = np.random.rand(len(df_1)) <= 0.92

In [29]:
# Creating dataframe of remaining 8% of random data
small = df_1[~red]

In [30]:
# Checking shape
small.shape

(71391, 10)

In [31]:
# Exporting small dataset
small.to_csv('small_data_for_dash.csv', index = False)